# Communications collectives

Les communications collectives peuvent faire :

* des **déplacements de données**
    * `MPI_Bcast`
    * `MPI_Scatter`
    * `MPI_Gather`, `MPI_Allgather`
    * `MPI_Alltoall`
* des **calculs collectifs**
    * `MPI_Reduce`, `MPI_Allreduce`

Chaque appel à ces fonctions doit être fait par
**tous les processus d'un même communicateur**.

## Déplacements de données

### Diffusion de données avec `MPI_Bcast`

Pour envoyer les mêmes informations à tous les processus d'un même
communicateur, on utilise une fonction effectuant une **diffusion** :

![Figure MPI_Bcast](images/mpi_bcast.svg)

[En **C**](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#section.6.4) :

```C
// int MPI_Bcast(void *tampon, int compte, MPI_Datatype type,
//               int racine, MPI_Comm comm)

ierr = MPI_Bcast(&a, 1, MPI_INT, 2, MPI_COMM_WORLD);

```

[En **Python**](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.bcast) :

```Python
# bcast(objet: Any, racine: int = 0) -> Any

a = comm.bcast(a, 2)
```

### Distribution de données avec `MPI_Scatter`

Pour envoyer une portion des données à chaque
processus d'un même communicateur, on utilise
une fonction effectuant une **distribution** :

![Figure MPI_Scatter](images/mpi_scatter.svg)

[En **C**](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#section.6.6) :

```C
// int MPI_Scatter(
//     void *envoi, int compte_envoi, MPI_Datatype type_envoi,
//     void *recep, int compte_recep, MPI_Datatype type_recep,
//     int racine, MPI_Comm comm)

ierr = MPI_Scatter( a, 1, MPI_INT,
                   &b, 1, MPI_INT, 2, MPI_COMM_WORLD);

```

[En **Python**](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.scatter) :

```Python
# scatter(envoi: Sequence[Any] | None, racine: int = 0) -> Any

b = comm.scatter(a, 2)
```

### Regroupement de données avec `MPI_Gather`

Pour récupérer plusieurs portions de données
dans un seul processus d'un communicateur, on
utilise une fonction effectuant un **regroupement** :

![Figure MPI_Gather](images/mpi_gather.svg)

[En **C**](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#section.6.5) :

```C
// int MPI_Gather(
//     void *envoi, int compte_envoi, MPI_Datatype type_envoi,
//     void *recep, int compte_recep, MPI_Datatype type_recep,
//     int racine, MPI_Comm comm)

ierr = MPI_Gather(&a, 1, MPI_INT,
                   b, 1, MPI_INT, 2, MPI_COMM_WORLD);

```

[En **Python**](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.gather) :

```Python
# gather(envoi: Any, racine: int = 0) -> list[Any] | None

b = comm.gather(a, 2)
```

### Regroupement à tous avec `MPI_Allgather`

C'est l'équivalent de `MPI_Gather` + `MPI_Bcast`,
mais en plus efficace :

![Figure MPI_Allgather](images/mpi_allgather.svg)

[En **C**](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#section.6.7) :

```C
// int MPI_Allgather(
//     void *envoi, int compte_envoi, MPI_Datatype type_envoi,
//     void *recep, int compte_recep, MPI_Datatype type_recep,
//     MPI_Comm comm)

ierr = MPI_Allgather(&a, 1, MPI_INT,
                      b, 1, MPI_INT, MPI_COMM_WORLD);

```

[En **Python**](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.allgather) :

```Python
# allgather(envoi: Any) -> list[Any]

b = comm.allgather(a)
```

### Transposition globale avec `MPI_Alltoall`

C'est l'équivalent de `MPI_Scatter` * `MPI_Gather`,
mais en plus efficace :

![Figure MPI_Alltoall](images/mpi_alltoall.svg)

[En **C**](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#section.6.8) :

```C
// int MPI_Alltoall(
//     void *envoi, int compte_envoi, MPI_Datatype type_envoi,
//     void *recep, int compte_recep, MPI_Datatype type_recep,
//     MPI_Comm comm)

ierr = MPI_Alltoall(a, 1, MPI_INT,
                    b, 1, MPI_INT, MPI_COMM_WORLD);

```

[En **Python**](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.alltoall) :

```Python
# alltoall(envoi: Sequence[Any]) -> list[Any]

b = comm.alltoall(a)
```

## Calculs collectifs

### Opérations de réduction

C'est l'équivalent d'un `MPI_Gather` avec une boucle effectuant une
opération de réduction. Voici quelques opérations de réduction :

Opération             | Constante MPI | Op([3, 5])
----------------------|---------------|-----------
Maximum               | `MPI_MAX`     | 5
Minimum               | `MPI_MIN`     | 3
Somme                 | `MPI_SUM`     | 8
Produit               | `MPI_PROD`    | 15
_ET_ logique          | `MPI_LAND`    | Vrai
_OU_ logique          | `MPI_LOR`     | Vrai
_OU exclusif_ logique | `MPI_LXOR`    | Faux
_ET_ binaire          | `MPI_BAND`    | 1 (001 = 011 & 101)
_OU_ binaire          | `MPI_BOR`     | 7 (111 = 011 \| 101)
_OU exclusif_ binaire | `MPI_BXOR`    | 6 (110 = 011 ^ 101)

### Réduction avec `MPI_Reduce`

Voici un exemple de réduction effectuant une somme :

![Figure MPI_Reduce](images/mpi_reduce.svg)

[En **C**](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#subsection.6.9.1) :

```C
// int MPI_Reduce(
//     void *envoi, void *recep, int compte, MPI_Datatype type,
//     MPI_Op op, int racine, MPI_Comm comm)

ierr = MPI_Reduce(&a, &b, 1, MPI_INT,
                  MPI_SUM, 2, MPI_COMM_WORLD);

```

[En **Python**](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.reduce) :

```Python
# reduce(envoi: Any, op: Op=SUM, racine: int = 0) -> Any | None

b = comm.reduce(a, MPI.SUM, 2)
```

### Réduction et diffusion avec `MPI_Allreduce`

C'est l'équivalent de `MPI_Reduce` + `MPI_Bcast`,
mais en plus efficace :

![Figure MPI_Allreduce](images/mpi_allreduce.svg)

[En **C**](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#subsection.6.9.6) :

```C
// int MPI_Allreduce(
//     void *envoi, void *recep, int compte, MPI_Datatype type,
//     MPI_Op op, MPI_Comm comm)

ierr = MPI_Allreduce(&a, &b, 1, MPI_INT,
                     MPI_SUM, MPI_COMM_WORLD);

```

[En **Python**](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.allreduce) :

```Python
# allreduce(envoi: Any, op: Op=SUM) -> Any

b = comm.allreduce(a, MPI.SUM)
```

## Division de l'espace de travail

* La stratégie la plus triviale consiste à diviser l'espace de
  travail en portions plus ou moins égales selon une dimension.
* Puisque la taille `N` d'une dimension n'est pas nécessairement
  un multiple de `size`, on ne peut pas faire une division entière
  de `N` par `size` pour définir une taille unique de portion.
  On risquerait alors d'oublier des éléments à calculer.
* Par contre, on peut intégrer `rank` et `rank + 1` dans le calcul
  des bornes inférieure et supérieure d'une portion de calcul,
  ce qui permet de gagner en résolution lors de la division entière
  tout en assurant le calcul global de `0` à `N - 1`, inclusivement :

En **C** :

```C
int debut = rank * N / size;
int fin = (rank + 1) * N / size
```

En **Python** :

```Python
debut = rank * N // size
fin = (rank + 1) * N // size
```

* Pour `rank == 0`, `debut` vaudra effectivement `0`
* La `fin` de `rank` est le `debut` de `rank + 1`
* Pour `rank == size - 1`, `fin` vaudra effectivement `N`
* Les itérations se font dans l'intervalle : `debut` <= `i` < `fin`

### Exemple - Intégration par la méthode des rectangles
Pour la fonction suivante :

$$f(x) = \sin^2(x) \ \mathrm{e}^{-x}$$

On souhaite calculer en parallèle une approximation de l'intégrale
$I$ de cette fonction dans l'intervalle allant de $0$ à $\pi$ :

$$I = \int_{0}^{\pi} \sin^2(x) \ \mathrm{e}^{-x} \mathrm{d}x
    = 2(1 - \mathrm{e}^{-\pi}) / 5$$

Pour ce faire, on calcule et additionne l'aire de $N$
rectangles de largeur $\Delta{}x = \pi / N$ et d'abscisse
$x_i = i\Delta{}x$, où $i$ varie de $0$ à $N - 1$ :

$$I \approx \Sigma_{i=0}^{N-1} \sin^2(x_i) \ \mathrm{e}^{-x_i} \Delta{}x$$
$$I \approx \Sigma_{i=0}^{N-1} \sin^2(i\Delta{}x) \ \mathrm{e}^{-i\Delta{}x} \Delta{}x$$

![Intégration par la méthode des rectangles, avec N = 10 rectangles](images/integration-rectangles.png)

Un **code séquentiel** en C aurait l'allure suivante :

```C
double integrale(int N)
{
    int i;
    int debut = 0;
    int fin = N;
    double somme = 0.0;
    const double dx = M_PI / N;

    for (i = debut; i < fin; i++) {
        double sinCarre = sin(i * dx) * sin(i * dx);
        somme += sinCarre * exp(-i * dx) * dx;
    }

    return somme;
}
```

Le même **code en parallèle avec MPI** aurait l'allure suivante :

```C
double integrale(int N, int rank, int size)
{
    int i;
    int debut = rank * N / size;
    int fin = (rank + 1) * N / size;
    double somme = 0.0, somme_locale = 0.0;
    const double dx = M_PI / N;

    for (i = debut; i < fin; i++) {
        double sinCarre = sin(i * dx) * sin(i * dx);
        somme_locale += sinCarre * exp(-i * dx) * dx;
    }

    MPI_Allreduce(&somme_locale, &somme, 1, MPI_DOUBLE,
                  MPI_SUM, MPI_COMM_WORLD);

    return somme;
}
```